In [1]:
%load_ext autoreload
%load_ext autotime

time: 212 µs (started: 2024-01-25 17:11:24 -08:00)


In [2]:
import os
import sys
wisio_dir = os.path.abspath("../")
sys.path.append(wisio_dir)
import warnings
warnings.filterwarnings('ignore')

time: 809 µs (started: 2024-01-25 17:11:24 -08:00)


In [3]:
import dask
import dask.dataframe as dd
import numpy as np
import pandas as pd
print('pandas', pd.__version__)
print('dask', dask.__version__)
print('numpy', np.__version__)

pandas 2.1.2
dask 2023.9.3
numpy 1.24.3
time: 1.65 s (started: 2024-01-25 17:11:24 -08:00)


In [4]:
# trace_path_pattern = '/usr/workspace/iopp/wisio_logs/dlp/dyad_dlio_unet3d_n2_p8_br1/*.pfw'
# trace_path_pattern = '/usr/workspace/iopp/wisio_logs/dlp/dyad_dlio_unet3d_n2_p8_br2/*.pfw'
trace_path_pattern = '/usr/workspace/iopp/wisio_logs/dlp/dyad_dlio_unet3d_n2_p8_br1-01-24/*.pfw'

time: 831 µs (started: 2024-01-25 17:11:26 -08:00)


In [5]:
%autoreload 2

from wisio.cluster_management import load_cluster_config
from wisio.dlp import DLPAnalyzer

cluster_config = load_cluster_config('cluster_config_local.yml')
cluster_config.local_dir = '/var/tmp/wisio-dlp-unet3d'

analyzer = DLPAnalyzer(
    checkpoint=False,
    checkpoint_dir='/usr/workspace/iopp/wisio_logs/_checkpoints/dlp_unet3d',
    cluster_config=cluster_config,
    working_dir='.wisio/dlp_unet3d',
    debug=True,
    verbose=False,
)

analyzer.cluster_manager.client.dashboard_link

[INFO] [17:11:27] Initializing DLP analyzer [/g/g91/izzet/projects/wisio/wisio/analyzer.py:100]
[DEBUG] [17:11:27] Using selector: EpollSelector [/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9/asyncio/selector_events.py:54]
[DEBUG] [17:11:30] Client dashboard at http://127.0.0.1:8787/status [/g/g91/izzet/projects/wisio/wisio/cluster_management.py:43]


'http://127.0.0.1:8787/status'

time: 3.88 s (started: 2024-01-25 17:11:26 -08:00)


In [6]:
traces = analyzer.read_pfw(trace_path_pattern=trace_path_pattern, time_granularity=1e6)
traces

,func_id,cat,pid,tid,ts,te,time,time_range,host_name,compute_time,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat
npartitions=498,,,,,,,,,,,,,,,,,,,,
,string,string,int64,int64,float64,float64,float64,float64,string,float64,float64,float64,float64,string,int16,int64,string,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 723 ms (started: 2024-01-25 16:05:28 -08:00)


In [7]:
traces.head()

,func_id,cat,pid,tid,ts,te,time,time_range,host_name,compute_time,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat
0,dyad_ucx_ep_cache_init,C_APP,727134,1454268,146.672930,146.672935,0.000005,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
1,ucx_allocate_buffer,C_APP,727134,1454268,146.672984,147.461877,0.788893,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
2,dyad_dtl_ucx_get_buffer,C_APP,727134,1454268,147.461942,147.461945,0.000003,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
3,ucx_connect,C_APP,727134,1454268,147.461958,147.498732,0.036774,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
4,dyad_ucx_request_init,C_APP,727134,1454268,147.498784,147.498786,0.000002,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0


time: 8.89 s (started: 2024-01-25 16:05:30 -08:00)


In [13]:
t = traces.compute()
t

,func_id,cat,pid,tid,ts,te,time,time_range,host_name,compute_time,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat
0,dyad_ucx_ep_cache_init,C_APP,727134,1454268,146.672930,146.672935,0.000005,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
1,ucx_allocate_buffer,C_APP,727134,1454268,146.672984,147.461877,0.788893,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
2,dyad_dtl_ucx_get_buffer,C_APP,727134,1454268,147.461942,147.461945,0.000003,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
3,ucx_connect,C_APP,727134,1454268,147.461958,147.498732,0.036774,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
4,dyad_ucx_request_init,C_APP,727134,1454268,147.498784,147.498786,0.000002,0.0,corona173,0.0,0.0,0.0,0.0,,0,0,app#corona173#727134#1454268,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,TorchFramework.checkpoint,ai_framework,4,3552674,191.801081,191.801124,0.000043,0.0,corona171,0.0,0.0,0.0,0.0,,0,0,app#corona171#4#3552674,0,1,0
351,DLIOBenchmark._train,dlio_benchmark,4,3552674,166.196360,191.802772,25.606412,0.0,corona171,0.0,0.0,0.0,0.0,,0,0,app#corona171#4#3552674,0,1,0
352,TorchFramework.get_loader,ai_framework,4,3552674,191.802932,191.802946,0.000014,0.0,corona171,0.0,0.0,0.0,0.0,,0,0,app#corona171#4#3552674,0,1,0
353,DyadTorchDataLoader.finalize,data_loader,4,3552674,191.802995,191.803002,0.000007,0.0,corona171,0.0,0.0,0.0,0.0,,0,0,app#corona171#4#3552674,0,1,0


time: 10.1 s (started: 2024-01-25 16:11:37 -08:00)


In [13]:
t.sort_values('time', ascending=False)

,func_id,cat,pid,tid,ts,te,time,time_range,host_name,compute_time,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat
1676,mod_main,C_APP,0,3552649,0,195357599,195357599,0.0,corona171,0,0,0,0,,0,0,app#corona171#0#3552649,0,1,0
2101,mod_main,C_APP,1,721415,594759,195546697,194951938,0.0,corona173,0,0,0,0,,0,0,app#corona173#1#721415,0,1,0
368,DLIOBenchmark.run,dlio_benchmark,0,3552666,42017140,192762765,150745625,42.0,corona171,0,0,0,0,,0,0,app#corona171#0#3552666,0,1,0
354,DLIOBenchmark.run,dlio_benchmark,7,3552680,42017143,192762589,150745446,42.0,corona171,0,0,0,0,,0,0,app#corona171#7#3552680,0,1,0
354,DLIOBenchmark.run,dlio_benchmark,4,3552674,42017141,192762572,150745431,42.0,corona171,0,0,0,0,,0,0,app#corona171#4#3552674,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,__fxstat64,POSIX,0,3554884,102687799,102687800,1,102.0,corona171,0,1,0,1,/l/ssd/haridev/dyad/img_011_of_168.npz,2,0,app#corona171#0#3554884,0,1,3
25,__fxstat64,POSIX,0,3554884,102077286,102077287,1,102.0,corona171,0,1,0,1,/l/ssd/haridev/dyad/img_13_of_16.npz,2,0,app#corona171#0#3554884,0,1,3
362,__fxstat64,POSIX,2,3559009,161015857,161015858,1,161.0,corona171,0,1,0,1,/l/ssd/haridev/dyad/img_016_of_168.npz,2,0,app#corona171#2#3559009,0,1,3
1272,__fxstat64,POSIX,2,3554822,103009238,103009239,1,103.0,corona171,0,1,0,1,/l/ssd/haridev/dyad/img_149_of_168.npz,2,0,app#corona171#2#3554822,0,1,3


time: 161 ms (started: 2024-01-25 15:58:56 -08:00)


In [15]:
list(t['func_id'].unique())

['dyad_ucx_ep_cache_init',
 'ucx_allocate_buffer',
 'dyad_dtl_ucx_get_buffer',
 'ucx_connect',
 'dyad_ucx_request_init',
 'ucx_send_no_wait',
 'dyad_send_callback',
 'ucx_recv_no_wait',
 'dyad_ucx_request_wait',
 'ucx_disconnect',
 'dyad_dtl_ucx_return_buffer',
 'ucx_warmup',
 'dyad_dtl_ucx_init',
 'dyad_dtl_init',
 'dyad_init',
 'gen_path_key',
 'dyad_kvs_read',
 'dyad_get_metadata',
 'dyad_excl_flock',
 'dyad_release_flock',
 'dyad_consume',
 'dyad_free_metadata',
 'NPZReader.__init__',
 'Dyad.__init__',
 'readlink',
 'Dyad.init',
 'Dyad.get_metadata',
 'dyad_open',
 'open',
 'fcntl',
 'lseek',
 'close',
 'Dyad.consume',
 'open64',
 '__fxstat64',
 'lseek64',
 'read',
 'Dyad.free_metadata',
 'DYADTorchDataset.__getitem__',
 '__xstat',
 'write',
 'fsync',
 'NPZReader.open',
 'NPZReader.get_sample',
 'FormatReader.preprocess',
 'NPZReader.close',
 'NPZReader.read_index',
 'Dyad.produce',
 'dyad_kvs_commit',
 'publish_via_flux',
 'dyad_commit',
 'dyad_produce',
 'dyad_fetch',
 'dyad_dtl_

time: 11.8 ms (started: 2024-01-25 16:12:00 -08:00)


In [6]:
%autoreload 2

result = analyzer.analyze_pfw(
    trace_path_pattern=trace_path_pattern, 
    metrics=['time'],
    view_types=['func_id', 'proc_name', 'time_range'],
    # slope_threshold=30,
    # exclude_bottlenecks=['excessive_metadata_access'],
    exclude_characteristics=['complexity'],
)

[INFO] [17:11:35] Read traces (0.417811743915081) [/g/g91/izzet/projects/wisio/wisio/dlp.py:196]
[INFO] [17:11:43] Compute high-level metrics (0.027844927040860057) [/g/g91/izzet/projects/wisio/wisio/analyzer.py:133]
[INFO] [17:11:45] Compute main view (0.14921783399768174) [/g/g91/izzet/projects/wisio/wisio/analyzer.py:146]
[INFO] [17:11:45] Compute metric boundaries (0.03678128798492253) [/g/g91/izzet/projects/wisio/wisio/analyzer.py:161]
[INFO] [17:11:52] Compute perspectives (1.5086448579095304) [/g/g91/izzet/projects/wisio/wisio/analyzer.py:185]
[INFO] [17:11:55] Detect I/O bottlenecks (0.4741328009404242) [/g/g91/izzet/projects/wisio/wisio/analyzer.py:199]
[DEBUG] [17:12:07] Detect I/O characteristics (12.686125766020268) [/g/g91/izzet/projects/wisio/wisio/analyzer.py:207]
[INFO] [17:12:18] Attach reasons to I/O bottlenecks (10.599900867091492) [/g/g91/izzet/projects/wisio/wisio/analyzer.py:213]


time: 43.1 s (started: 2024-01-25 17:11:35 -08:00)


In [14]:
result.main_view.head()

time  count  size_min  size_max  \
func_id       proc_name                                                        
<module>.iter app#corona171#0#3552666  119.690100      5         0         0   
              app#corona171#1#3552668  118.566421      5         0         0   
              app#corona171#2#3552670  118.162860      5         0         0   
              app#corona171#3#3552672  119.018237      5         0         0   
              app#corona171#4#3552674  118.949583      5         0         0   

                                       size  read_time  write_time  \
func_id       proc_name                                              
<module>.iter app#corona171#0#3552666     0        0.0         0.0   
              app#corona171#1#3552668     0        0.0         0.0   
              app#corona171#2#3552670     0        0.0         0.0   
              app#corona171#3#3552672     0        0.0         0.0   
              app#corona171#4#3552674     0        0.0         0.0   

                                       metadata_time  read_size  write_size  \
func_id       proc_name                                                       
<module>.iter app#corona171#0#3552666            0.0          0           0   
              app#corona171#1#3552668            0.0          0           0   
              app#corona171#2#3552670            0.0          0           0   
              app#corona171#3#3552672            0.0          0           0   
              app#corona171#4#3552674            0.0          0           0   

                                       ...  seek_time  stat_time  close_count  \
func_id       proc_name                ...                                      
<module>.iter app#corona171#0#3552666  ...        0.0        0.0            0   
              app#corona171#1#3552668  ...        0.0        0.0            0   
              app#corona171#2#3552670  ...        0.0        0.0            0   
              app#corona171#3#3552672  ...        0.0        0.0            0   
              app#corona171#4#3552674  ...        0.0        0.0            0   

                                       open_count  seek_count  stat_count  \
func_id       proc_name                                                     
<module>.iter app#corona171#0#3552666           0           0           0   
              app#corona171#1#3552668           0           0           0   
              app#corona171#2#3552670           0           0           0   
              app#corona171#3#3552672           0           0           0   
              app#corona171#4#3552674           0           0           0   

                                        bw  iops  intensity  \
func_id       proc_name                                       
<module>.iter app#corona171#0#3552666  0.0   0.0        0.0   
              app#corona171#1#3552668  0.0   0.0        0.0   
              app#corona171#2#3552670  0.0   0.0        0.0   
              app#corona171#3#3552672  0.0   0.0        0.0   
              app#corona171#4#3552674  0.0   0.0        0.0   

                                                        id  
func_id       proc_name                                     
<module>.iter app#corona171#0#3552666  6137882666969451002  
              app#corona171#1#3552668   121123674029208365  
              app#corona171#2#3552670 -2806873664195054869  
              app#corona171#3#3552672  7882268262537317835  
              app#corona171#4#3552674 -1068830089011103764  

[5 rows x 41 columns]

time: 60.4 ms (started: 2024-01-25 16:30:45 -08:00)


In [7]:
mv4 = result.main_view.compute()
mv4

time  count  size_min  \
func_id       proc_name               time_range                                
<module>.iter app#corona171#0#4081389 0.0         101.815812      5         0   
              app#corona171#1#4081391 0.0          99.792792      5         0   
              app#corona171#2#4081393 0.0         102.356178      5         0   
              app#corona171#3#4081395 0.0         102.258411      5         0   
              app#corona171#4#4081397 0.0         102.872272      5         0   
...                                                      ...    ...       ...   
write         app#corona171#7#4082624 0.0           0.007362     10        55   
              app#corona171#7#4082946 0.0           0.003734     13        55   
              app#corona171#7#4083586 0.0           0.027259     18        55   
              app#corona171#7#4084359 0.0           0.000809      1   2096960   
              app#corona171#7#4085338 0.0           0.000267      3        13   

                                                  size_max      size  \
func_id       proc_name               time_range                       
<module>.iter app#corona171#0#4081389 0.0                0         0   
              app#corona171#1#4081391 0.0                0         0   
              app#corona171#2#4081393 0.0                0         0   
              app#corona171#3#4081395 0.0                0         0   
              app#corona171#4#4081397 0.0                0         0   
...                                                    ...       ...   
write         app#corona171#7#4082624 0.0          2096960   8387950   
              app#corona171#7#4082946 0.0          2096960   8388005   
              app#corona171#7#4083586 0.0          2096960  12581980   
              app#corona171#7#4084359 0.0          2096960   2096960   
              app#corona171#7#4085338 0.0               35        68   

                                                  read_time  write_time  \
func_id       proc_name               time_range                          
<module>.iter app#corona171#0#4081389 0.0               0.0    0.000000   
              app#corona171#1#4081391 0.0               0.0    0.000000   
              app#corona171#2#4081393 0.0               0.0    0.000000   
              app#corona171#3#4081395 0.0               0.0    0.000000   
              app#corona171#4#4081397 0.0               0.0    0.000000   
...                                                     ...         ...   
write         app#corona171#7#4082624 0.0               0.0    0.007362   
              app#corona171#7#4082946 0.0               0.0    0.003734   
              app#corona171#7#4083586 0.0               0.0    0.027259   
              app#corona171#7#4084359 0.0               0.0    0.000809   
              app#corona171#7#4085338 0.0               0.0    0.000267   

                                                  metadata_time  read_size  \
func_id       proc_name               time_range                             
<module>.iter app#corona171#0#4081389 0.0                   0.0          0   
              app#corona171#1#4081391 0.0                   0.0          0   
              app#corona171#2#4081393 0.0                   0.0          0   
              app#corona171#3#4081395 0.0                   0.0          0   
              app#corona171#4#4081397 0.0                   0.0          0   
...                                                         ...        ...   
write         app#corona171#7#4082624 0.0                   0.0          0   
              app#corona171#7#4082946 0.0                   0.0          0   
              app#corona171#7#4083586 0.0                   0.0          0   
              app#corona171#7#4084359 0.0                   0.0          0   
              app#corona171#7#4085338 0.0                   0.0          0   

                                                  write_size  ...  seek_ti

time: 67 ms (started: 2024-01-25 17:13:12 -08:00)


In [8]:
mv4.query("func_id == 'dyad_fetch_request_cb' | func_id == 'dyad_get_data'")

time  \
func_id               proc_name                     time_range             
dyad_fetch_request_cb app#corona171#0#4081372       0.0         0.852878   
                      app#corona173#1#2394207       0.0         1.012122   
dyad_get_data         app#corona171#4081764#8163528 0.0         0.162549   
                      app#corona171#4081765#8163530 0.0         0.005252   
                      app#corona171#4081766#8163532 0.0         0.063133   
...                                                                  ...   
                      app#corona173#2397113#4794226 0.0         0.024165   
                      app#corona173#2397177#4794354 0.0         0.022039   
                      app#corona173#2397241#4794482 0.0         0.022030   
                      app#corona173#2397305#4794610 0.0         0.019990   
                      app#corona173#2397369#4794738 0.0         0.051690   

                                                                count  \
func_id               proc_name                     time_range          
dyad_fetch_request_cb app#corona171#0#4081372       0.0            56   
                      app#corona173#1#2394207       0.0            67   
dyad_get_data         app#corona171#4081764#8163528 0.0             2   
                      app#corona171#4081765#8163530 0.0             1   
                      app#corona171#4081766#8163532 0.0             3   
...                                                               ...   
                      app#corona173#2397113#4794226 0.0             1   
                      app#corona173#2397177#4794354 0.0             1   
                      app#corona173#2397241#4794482 0.0             2   
                      app#corona173#2397305#4794610 0.0             1   
                      app#corona173#2397369#4794738 0.0             1   

                                                                size_min  \
func_id               proc_name                     time_range             
dyad_fetch_request_cb app#corona171#0#4081372       0.0                0   
                      app#corona173#1#2394207       0.0                0   
dyad_get_data         app#corona171#4081764#8163528 0.0                0   
                      app#corona171#4081765#8163530 0.0                0   
                      app#corona171#4081766#8163532 0.0                0   
...                                                                  ...   
                      app#corona173#2397113#4794226 0.0                0   
                      app#corona173#2397177#4794354 0.0                0   
                      app#corona173#2397241#4794482 0.0                0   
                      app#corona173#2397305#4794610 0.0                0   
                      app#corona173#2397369#4794738 0.0                0   

                                                                size_max  \
func_id               proc_name                     time_range             
dyad_fetch_request_cb app#corona171#0#4081372       0.0                0   
                      app#corona173#1#2394207       0.0                0   
dyad_get_data         app#corona171#4081764#8163528 0.0                0   
                      app#corona171#4081765#8163530 0.0                0   
                      app#corona171#4081766#8163532 0.0                0   
...                                                                  ...   
                      app#corona173#2397113#4794226 0.0                0   
                      app#corona173#2397177#4794354 0.0                0   
                      app#corona173#2397241#4794482 0.0                0   
                      app#corona173#2397305#4794610 0.0                0   
                      app#corona173#2397369#4794738 0.0                0   

                                                                size  \
func_id               proc_name                     time_range         
dyad_fe

time: 53.1 ms (started: 2024-01-25 17:13:29 -08:00)


In [15]:
# proc_names = mv4.reset_index().query('func_id == "dyad_fetch_request_cb"')['proc_name'].unique()
mv4.query(f"(proc_name == 'app#corona171#0#4081372' | proc_name == 'app#corona173#1#2394207') & (func_id == 'dyad_dtl_ucx_init')")

time  count  \
func_id           proc_name               time_range                    
dyad_dtl_ucx_init app#corona171#0#4081372 0.0         0.438014      1   
                  app#corona173#1#2394207 0.0         0.436600      1   

                                                      size_min  size_max  \
func_id           proc_name               time_range                       
dyad_dtl_ucx_init app#corona171#0#4081372 0.0                0         0   
                  app#corona173#1#2394207 0.0                0         0   

                                                      size  read_time  \
func_id           proc_name               time_range                    
dyad_dtl_ucx_init app#corona171#0#4081372 0.0            0        0.0   
                  app#corona173#1#2394207 0.0            0        0.0   

                                                      write_time  \
func_id           proc_name               time_range               
dyad_dtl_ucx_init app#corona171#0#4081372 0.0                0.0   
                  app#corona173#1#2394207 0.0                0.0   

                                                      metadata_time  \
func_id           proc_name               time_range                  
dyad_dtl_ucx_init app#corona171#0#4081372 0.0                   0.0   
                  app#corona173#1#2394207 0.0                   0.0   

                                                      read_size  write_size  \
func_id           proc_name               time_range                          
dyad_dtl_ucx_init app#corona171#0#4081372 0.0                 0           0   
                  app#corona173#1#2394207 0.0                 0           0   

                                                      ...  seek_time  \
func_id           proc_name               time_range  ...              
dyad_dtl_ucx_init app#corona171#0#4081372 0.0         ...        0.0   
                  app#corona173#1#2394207 0.0         ...        0.0   

                                                      stat_time  close_count  \
func_id           proc_name               time_range                           
dyad_dtl_ucx_init app#corona171#0#4081372 0.0               0.0            0   
                  app#corona173#1#2394207 0.0               0.0            0   

                                                      open_count  seek_count  \
func_id           proc_name               time_range                           
dyad_dtl_ucx_init app#corona171#0#4081372 0.0                  0           0   
                  app#corona173#1#2394207 0.0                  0           0   

                                                      stat_count   bw  iops  \
func_id           proc_name               time_range                          
dyad_dtl_ucx_init app#corona171#0#4081372 0.0                  0  0.0   0.0   
                  app#corona173#1#2394207 0.0                  0  0.0   0.0   

                                                      intensity  \
func_id           proc_name               time_range              
dyad_dtl_ucx_init app#corona171#0#4081372 0.0               0.0   
                  app#corona173#1#2394207 0.0               0.0   

                                                                       id  
func_id           proc_name               time_range                       
dyad_dtl_ucx_init app#corona171#0#4081372 0.0         8038504454541040666  
                  app#corona173#1#2394207 0.0        -9132465457416169451  

[2 rows x 41 columns]

time: 33.9 ms (started: 2024-01-25 17:17:53 -08:00)


In [18]:
# app#corona171#0#4101283, app#corona171#1#4101288, app#corona173#2#2413354, app#corona173#3#2413358	
# func_id == 'dyad_fetch_request_cb' | func_id == 'dyad_get_data'
mv4.query("(func_id == 'dyad_dtl_ucx_init')")

time  count  \
func_id           proc_name                     time_range                    
dyad_dtl_ucx_init app#corona171#0#4081372       0.0         0.438014      1   
                  app#corona171#4081764#8163528 0.0         0.749078      1   
                  app#corona171#4081765#8163530 0.0         0.748916      1   
                  app#corona171#4081766#8163532 0.0         0.770547      1   
                  app#corona171#4081767#8163534 0.0         0.753705      1   
...                                                              ...    ...   
                  app#corona173#2397113#4794226 0.0         2.303945      1   
                  app#corona173#2397177#4794354 0.0         2.878001      1   
                  app#corona173#2397241#4794482 0.0         2.038789      1   
                  app#corona173#2397305#4794610 0.0         2.142953      1   
                  app#corona173#2397369#4794738 0.0         2.349583      1   

                                                            size_min  \
func_id           proc_name                     time_range             
dyad_dtl_ucx_init app#corona171#0#4081372       0.0                0   
                  app#corona171#4081764#8163528 0.0                0   
                  app#corona171#4081765#8163530 0.0                0   
                  app#corona171#4081766#8163532 0.0                0   
                  app#corona171#4081767#8163534 0.0                0   
...                                                              ...   
                  app#corona173#2397113#4794226 0.0                0   
                  app#corona173#2397177#4794354 0.0                0   
                  app#corona173#2397241#4794482 0.0                0   
                  app#corona173#2397305#4794610 0.0                0   
                  app#corona173#2397369#4794738 0.0                0   

                                                            size_max  size  \
func_id           proc_name                     time_range                   
dyad_dtl_ucx_init app#corona171#0#4081372       0.0                0     0   
                  app#corona171#4081764#8163528 0.0                0     0   
                  app#corona171#4081765#8163530 0.0                0     0   
                  app#corona171#4081766#8163532 0.0                0     0   
                  app#corona171#4081767#8163534 0.0                0     0   
...                                                              ...   ...   
                  app#corona173#2397113#4794226 0.0                0     0   
                  app#corona173#2397177#4794354 0.0                0     0   
                  app#corona173#2397241#4794482 0.0                0     0   
                  app#corona173#2397305#4794610 0.0                0     0   
                  app#corona173#2397369#4794738 0.0                0     0   

                                                            read_time  \
func_id           proc_name                     time_range              
dyad_dtl_ucx_init app#corona171#0#4081372       0.0               0.0   
                  app#corona171#4081764#8163528 0.0               0.0   
                  app#corona171#4081765#8163530 0.0               0.0   
                  app#corona171#4081766#8163532 0.0               0.0   
                  app#corona171#4081767#8163534 0.0               0.0   
...                                                               ...   
                  app#corona173#2397113#4794226 0.0               0.0   
                  app#corona173#2397177#4794354 0.0               0.0   
                  app#corona173#2397241#4794482 0.0               0.0   
                  app#corona173#2397305#4794610 0.0               0.0   
                  app#corona173#2397369#4794738 0.0               0.0   

                                                            write_time  \
func_id           proc_name                     time_range  

time: 53.3 ms (started: 2024-01-25 17:21:39 -08:00)


In [7]:
mv3 = result.main_view.compute()
mv3

time  count  size_min  size_max  \
func_id       proc_name                                                        
<module>.iter app#corona171#0#4081389  101.815812      5         0         0   
              app#corona171#1#4081391   99.792792      5         0         0   
              app#corona171#2#4081393  102.356178      5         0         0   
              app#corona171#3#4081395  102.258411      5         0         0   
              app#corona171#4#4081397  102.872272      5         0         0   
...                                           ...    ...       ...       ...   
write         app#corona171#7#4082624    0.007362     10        55   2096960   
              app#corona171#7#4082946    0.003734     13        55   2096960   
              app#corona171#7#4083586    0.027259     18        55   2096960   
              app#corona171#7#4084359    0.000809      1   2096960   2096960   
              app#corona171#7#4085338    0.000267      3        13        35   

                                           size  read_time  write_time  \
func_id       proc_name                                                  
<module>.iter app#corona171#0#4081389         0        0.0    0.000000   
              app#corona171#1#4081391         0        0.0    0.000000   
              app#corona171#2#4081393         0        0.0    0.000000   
              app#corona171#3#4081395         0        0.0    0.000000   
              app#corona171#4#4081397         0        0.0    0.000000   
...                                         ...        ...         ...   
write         app#corona171#7#4082624   8387950        0.0    0.007362   
              app#corona171#7#4082946   8388005        0.0    0.003734   
              app#corona171#7#4083586  12581980        0.0    0.027259   
              app#corona171#7#4084359   2096960        0.0    0.000809   
              app#corona171#7#4085338        68        0.0    0.000267   

                                       metadata_time  read_size  write_size  \
func_id       proc_name                                                       
<module>.iter app#corona171#0#4081389            0.0          0           0   
              app#corona171#1#4081391            0.0          0           0   
              app#corona171#2#4081393            0.0          0           0   
              app#corona171#3#4081395            0.0          0           0   
              app#corona171#4#4081397            0.0          0           0   
...                                              ...        ...         ...   
write         app#corona171#7#4082624            0.0          0     8387950   
              app#corona171#7#4082946            0.0          0     8388005   
              app#corona171#7#4083586            0.0          0    12581980   
              app#corona171#7#4084359            0.0          0     2096960   
              app#corona171#7#4085338            0.0          0          68   

                                       ...  seek_time  stat_time  close_count  \
func_id       proc_name                ...                                      
<module>.iter app#corona171#0#4081389  ...        0.0        0.0            0   
              app#corona171#1#4081391  ...        0.0        0.0            0   
              app#corona171#2#4081393  ...        0.0        0.0            0   
              app#corona171#3#4081395  ...        0.0        0.0            0   
              app#corona171#4#4081397  ...        0.0        0.0            0   
...                                    ...        ...        ...          ...   
write         app#corona171#7#4082624  ...        0.0        0.0            0   
              app#corona171#7#4082946  ...        0.0        0.0            0   
              app#corona171#7#4083586  ...        0.0        0.0            0   
              app#corona171#7#4084359  ...        0.0        0.0            0   
              app#corona171#7#4085338  ..

time: 78 ms (started: 2024-01-25 16:59:12 -08:00)


In [11]:
mv3.query("func_id == 'dyad_fetch_request_cb' | func_id == 'dyad_get_data'")

time  count  \
func_id               proc_name                                        
dyad_fetch_request_cb app#corona171#0#4081372        0.852878     56   
                      app#corona173#1#2394207        1.012122     67   
dyad_get_data         app#corona171#4081764#8163528  0.162549      2   
                      app#corona171#4081765#8163530  0.005252      1   
                      app#corona171#4081766#8163532  0.063133      3   
...                                                       ...    ...   
                      app#corona173#2397113#4794226  0.024165      1   
                      app#corona173#2397177#4794354  0.022039      1   
                      app#corona173#2397241#4794482  0.022030      2   
                      app#corona173#2397305#4794610  0.019990      1   
                      app#corona173#2397369#4794738  0.051690      1   

                                                     size_min  size_max  size  \
func_id               proc_name                                                 
dyad_fetch_request_cb app#corona171#0#4081372               0         0     0   
                      app#corona173#1#2394207               0         0     0   
dyad_get_data         app#corona171#4081764#8163528         0         0     0   
                      app#corona171#4081765#8163530         0         0     0   
                      app#corona171#4081766#8163532         0         0     0   
...                                                       ...       ...   ...   
                      app#corona173#2397113#4794226         0         0     0   
                      app#corona173#2397177#4794354         0         0     0   
                      app#corona173#2397241#4794482         0         0     0   
                      app#corona173#2397305#4794610         0         0     0   
                      app#corona173#2397369#4794738         0         0     0   

                                                     read_time  write_time  \
func_id               proc_name                                              
dyad_fetch_request_cb app#corona171#0#4081372              0.0         0.0   
                      app#corona173#1#2394207              0.0         0.0   
dyad_get_data         app#corona171#4081764#8163528        0.0         0.0   
                      app#corona171#4081765#8163530        0.0         0.0   
                      app#corona171#4081766#8163532        0.0         0.0   
...                                                        ...         ...   
                      app#corona173#2397113#4794226        0.0         0.0   
                      app#corona173#2397177#4794354        0.0         0.0   
                      app#corona173#2397241#4794482        0.0         0.0   
                      app#corona173#2397305#4794610        0.0         0.0   
                      app#corona173#2397369#4794738        0.0         0.0   

                                                     metadata_time  read_size  \
func_id               proc_name                                                 
dyad_fetch_request_cb app#corona171#0#4081372                  0.0          0   
                      app#corona173#1#2394207                  0.0          0   
dyad_get_data         app#corona171#4081764#8163528            0.0          0   
                      app#corona171#4081765#8163530            0.0          0   
                      app#corona171#4081766#8163532            0.0          0   
...                                                            ...        ...   
                      app#corona173#2397113#4794226            0.0          0   
                      app#corona173#2397177#4794354            0.0          0   
                      app#corona173#2397241#4794482            0.0          0   
                      app#corona173#2397305#4794610            0.0          0   
                      app#corona173#2397369#4794738            0.0          0  

time: 50.3 ms (started: 2024-01-25 17:06:26 -08:00)


In [12]:
mv3.query("proc_name == 'app#corona171#0#4081372' & func_id == 'dyad_dtl_ucx_init'")

,,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,seek_time,stat_time,close_count,open_count,seek_count,stat_count,bw,iops,intensity,id
func_id,proc_name,,,,,,,,,,,,,,,,,,,,,
dyad_dtl_ucx_init,app#corona171#0#4081372,0.438014,1,0,0,0,0.0,0.0,0.0,0,0,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,-4223373360686424256


time: 33.2 ms (started: 2024-01-25 17:07:03 -08:00)


view_types = x + proc_name + time_range 

remove proc_name = max
remove time_range = sum



In [9]:
t3 = mv3.groupby('func_id')['time'].max().to_frame().sort_values('time', ascending=False).head(50)
t3.reset_index().sort_values('func_id')

,func_id,time
2,<module>.iter,102.872272
4,<module>.yield,10.245145
25,ConfigArguments.derive_configurations,0.425089
1,DLIOBenchmark._train,117.984884
23,DLIOBenchmark.initialize,0.504215
0,DLIOBenchmark.run,118.031157
5,DYADTorchDataset.__getitem__,8.871515
16,Dyad.__init__,0.879297
27,Dyad.consume_w_metadata,0.233706
48,Dyad.free_metadata,0.050293


time: 17 ms (started: 2024-01-25 16:59:39 -08:00)


In [15]:
mv = result.main_view.compute()
mv

time  count  size_min  size_max  \
func_id       proc_name                                                        
<module>.iter app#corona171#0#3552666  119.690100      5         0         0   
              app#corona171#1#3552668  118.566421      5         0         0   
              app#corona171#2#3552670  118.162860      5         0         0   
              app#corona171#3#3552672  119.018237      5         0         0   
              app#corona171#4#3552674  118.949583      5         0         0   
...                                           ...    ...       ...       ...   
write         app#corona173#9#721814     0.026591     24        55   2096704   
              app#corona173#9#721822     0.006333     22        55   2096960   
              app#corona173#9#722390     0.001494      8        55   2096704   
              app#corona173#9#723593     0.001506      2   2096960   2096960   
              app#corona173#9#724163     0.001087      1   2096960   2096960   

                                           size  read_time  write_time  \
func_id       proc_name                                                  
<module>.iter app#corona171#0#3552666         0        0.0    0.000000   
              app#corona171#1#3552668         0        0.0    0.000000   
              app#corona171#2#3552670         0        0.0    0.000000   
              app#corona171#3#3552672         0        0.0    0.000000   
              app#corona171#4#3552674         0        0.0    0.000000   
...                                         ...        ...         ...   
write         app#corona173#9#721814   12582090        0.0    0.026591   
              app#corona173#9#721822   14678995        0.0    0.006333   
              app#corona173#9#722390    4194030        0.0    0.001494   
              app#corona173#9#723593    4193920        0.0    0.001506   
              app#corona173#9#724163    2096960        0.0    0.001087   

                                       metadata_time  read_size  write_size  \
func_id       proc_name                                                       
<module>.iter app#corona171#0#3552666            0.0          0           0   
              app#corona171#1#3552668            0.0          0           0   
              app#corona171#2#3552670            0.0          0           0   
              app#corona171#3#3552672            0.0          0           0   
              app#corona171#4#3552674            0.0          0           0   
...                                              ...        ...         ...   
write         app#corona173#9#721814             0.0          0    12582090   
              app#corona173#9#721822             0.0          0    14678995   
              app#corona173#9#722390             0.0          0     4194030   
              app#corona173#9#723593             0.0          0     4193920   
              app#corona173#9#724163             0.0          0     2096960   

                                       ...  seek_time  stat_time  close_count  \
func_id       proc_name                ...                                      
<module>.iter app#corona171#0#3552666  ...        0.0        0.0            0   
              app#corona171#1#3552668  ...        0.0        0.0            0   
              app#corona171#2#3552670  ...        0.0        0.0            0   
              app#corona171#3#3552672  ...        0.0        0.0            0   
              app#corona171#4#3552674  ...        0.0        0.0            0   
...                                    ...        ...        ...          ...   
write         app#corona173#9#721814   ...        0.0        0.0            0   
              app#corona173#9#721822   ...        0.0        0.0            0   
              app#corona173#9#722390   ...        0.0        0.0            0   
              app#corona173#9#723593   ...        0.0        0.0            0   
              app#corona173#9#724163   ..

time: 79.3 ms (started: 2024-01-25 16:31:00 -08:00)


In [8]:
mv2.query("func_id == 'dyad_fetch_request_cb' | func_id == 'dyad_get_data'")



time  count  \
func_id               proc_name                                        
dyad_fetch_request_cb app#corona171#0#4101283        0.581338     21   
                      app#corona171#1#4101288        0.834242     24   
                      app#corona173#2#2413354        0.830964     39   
                      app#corona173#3#2413358        1.126275     45   
dyad_get_data         app#corona171#4101737#8203474  0.044950      2   
...                                                       ...    ...   
                      app#corona173#2416123#4832246  0.087395      2   
                      app#corona173#2416187#4832374  0.009831      1   
                      app#corona173#2416251#4832502  0.030224      1   
                      app#corona173#2416443#4832886  0.052994      2   
                      app#corona173#2416507#4833014  0.037332      1   

                                                     size_min  size_max  size  \
func_id               proc_name                                                 
dyad_fetch_request_cb app#corona171#0#4101283               0         0     0   
                      app#corona171#1#4101288               0         0     0   
                      app#corona173#2#2413354               0         0     0   
                      app#corona173#3#2413358               0         0     0   
dyad_get_data         app#corona171#4101737#8203474         0         0     0   
...                                                       ...       ...   ...   
                      app#corona173#2416123#4832246         0         0     0   
                      app#corona173#2416187#4832374         0         0     0   
                      app#corona173#2416251#4832502         0         0     0   
                      app#corona173#2416443#4832886         0         0     0   
                      app#corona173#2416507#4833014         0         0     0   

                                                     read_time  write_time  \
func_id               proc_name                                              
dyad_fetch_request_cb app#corona171#0#4101283              0.0         0.0   
                      app#corona171#1#4101288              0.0         0.0   
                      app#corona173#2#2413354              0.0         0.0   
                      app#corona173#3#2413358              0.0         0.0   
dyad_get_data         app#corona171#4101737#8203474        0.0         0.0   
...                                                        ...         ...   
                      app#corona173#2416123#4832246        0.0         0.0   
                      app#corona173#2416187#4832374        0.0         0.0   
                      app#corona173#2416251#4832502        0.0         0.0   
                      app#corona173#2416443#4832886        0.0         0.0   
                      app#corona173#2416507#4833014        0.0         0.0   

                                                     metadata_time  read_size  \
func_id               proc_name                                                 
dyad_fetch_request_cb app#corona171#0#4101283                  0.0          0   
                      app#corona171#1#4101288                  0.0          0   
                      app#corona173#2#2413354                  0.0          0   
                      app#corona173#3#2413358                  0.0          0   
dyad_get_data         app#corona171#4101737#8203474            0.0          0   
...                                                            ...        ...   
                      app#corona173#2416123#4832246            0.0          0   
                      app#corona173#2416187#4832374            0.0          0   
                      app#corona173#2416251#4832502            0.0          0   
                      app#corona173#2416443#4832886            0.0          0   
                      app#corona173#2416507#4833014            0.0          0  

time: 54.1 ms (started: 2024-01-25 16:46:55 -08:00)


In [21]:
mv.query("func_id == 'dyad_fetch_request_cb' | func_id == 'dyad_get_data'")

time  count  \
func_id               proc_name                                        
dyad_fetch_request_cb app#corona171#0#3552649        1.455258     65   
                      app#corona173#1#721415         1.956129     83   
dyad_get_data         app#corona171#3553029#7106058  0.065596      3   
                      app#corona171#3553030#7106060  0.023735      2   
                      app#corona171#3553031#7106062  0.014278      1   
...                                                       ...    ...   
                      app#corona173#725399#1450798   0.007564      1   
                      app#corona173#725931#1451862   0.025399      1   
                      app#corona173#725933#1451866   0.006178      1   
                      app#corona173#726209#1452418   0.005169      1   
                      app#corona173#727835#1455670   0.032624      1   

                                                     size_min  size_max  size  \
func_id               proc_name                                                 
dyad_fetch_request_cb app#corona171#0#3552649               0         0     0   
                      app#corona173#1#721415                0         0     0   
dyad_get_data         app#corona171#3553029#7106058         0         0     0   
                      app#corona171#3553030#7106060         0         0     0   
                      app#corona171#3553031#7106062         0         0     0   
...                                                       ...       ...   ...   
                      app#corona173#725399#1450798          0         0     0   
                      app#corona173#725931#1451862          0         0     0   
                      app#corona173#725933#1451866          0         0     0   
                      app#corona173#726209#1452418          0         0     0   
                      app#corona173#727835#1455670          0         0     0   

                                                     read_time  write_time  \
func_id               proc_name                                              
dyad_fetch_request_cb app#corona171#0#3552649              0.0         0.0   
                      app#corona173#1#721415               0.0         0.0   
dyad_get_data         app#corona171#3553029#7106058        0.0         0.0   
                      app#corona171#3553030#7106060        0.0         0.0   
                      app#corona171#3553031#7106062        0.0         0.0   
...                                                        ...         ...   
                      app#corona173#725399#1450798         0.0         0.0   
                      app#corona173#725931#1451862         0.0         0.0   
                      app#corona173#725933#1451866         0.0         0.0   
                      app#corona173#726209#1452418         0.0         0.0   
                      app#corona173#727835#1455670         0.0         0.0   

                                                     metadata_time  read_size  \
func_id               proc_name                                                 
dyad_fetch_request_cb app#corona171#0#3552649                  0.0          0   
                      app#corona173#1#721415                   0.0          0   
dyad_get_data         app#corona171#3553029#7106058            0.0          0   
                      app#corona171#3553030#7106060            0.0          0   
                      app#corona171#3553031#7106062            0.0          0   
...                                                            ...        ...   
                      app#corona173#725399#1450798             0.0          0   
                      app#corona173#725931#1451862             0.0          0   
                      app#corona173#725933#1451866             0.0          0   
                      app#corona173#726209#1452418             0.0          0   
                      app#corona173#727835#1455670             0.0          0  

time: 55.3 ms (started: 2024-01-25 16:42:44 -08:00)


In [10]:
t2 = mv2.groupby('func_id')['time'].max().to_frame().sort_values('time', ascending=False).head(50)
t2

,time
func_id,
DLIOBenchmark.run,279.214226
DLIOBenchmark._train,278.906556
<module>.iter,263.344462
<module>.yield,17.363886
DYADTorchDataset.__getitem__,16.597976
TorchFramework.compute,13.629963
NPZReader.read_index,13.150801
NPZReader.open,12.872611
read,6.275254


time: 14.7 ms (started: 2024-01-25 16:50:14 -08:00)


In [20]:
mv.groupby('func_id')['time'].max().to_frame().sort_values('time', ascending=False).head(50)

,time
func_id,
DLIOBenchmark.run,150.745625
DLIOBenchmark._train,136.150694
<module>.iter,120.516533
TorchFramework.init_loader,14.667664
TorchFramework.compute,13.620230
<module>.yield,11.085548
DYADTorchDataset.__getitem__,6.381378
NPZReader.read_index,6.186760
NPZReader.open,6.036863


time: 13.2 ms (started: 2024-01-25 16:32:54 -08:00)


In [7]:
mv2 = result.main_view.compute()
mv2

time  count  size_min  size_max  \
func_id       proc_name                                                        
<module>.iter app#corona171#0#4101311  257.433828      5         0         0   
              app#corona171#1#4101314  257.240971      5         0         0   
              app#corona171#2#4101317  261.694590      5         0         0   
              app#corona171#3#4101319  258.969103      5         0         0   
              app#corona171#4#4101314  254.825172      5         0         0   
...                                           ...    ...       ...       ...   
write         app#corona173#9#2414020    0.020369     13        55   2096960   
              app#corona173#9#2414745    0.000705      4        55   2096704   
              app#corona173#9#2415364    0.004222     13        55   2096960   
              app#corona173#9#2415823    0.002975     13        55   2096960   
              app#corona173#9#2416516    0.000923      1   2096960   2096960   

                                          size  read_time  write_time  \
func_id       proc_name                                                 
<module>.iter app#corona171#0#4101311        0        0.0    0.000000   
              app#corona171#1#4101314        0        0.0    0.000000   
              app#corona171#2#4101317        0        0.0    0.000000   
              app#corona171#3#4101319        0        0.0    0.000000   
              app#corona171#4#4101314        0        0.0    0.000000   
...                                        ...        ...         ...   
write         app#corona173#9#2414020  8388005        0.0    0.020369   
              app#corona173#9#2414745  2097015        0.0    0.000705   
              app#corona173#9#2415364  8388005        0.0    0.004222   
              app#corona173#9#2415823  8388005        0.0    0.002975   
              app#corona173#9#2416516  2096960        0.0    0.000923   

                                       metadata_time  read_size  write_size  \
func_id       proc_name                                                       
<module>.iter app#corona171#0#4101311            0.0          0           0   
              app#corona171#1#4101314            0.0          0           0   
              app#corona171#2#4101317            0.0          0           0   
              app#corona171#3#4101319            0.0          0           0   
              app#corona171#4#4101314            0.0          0           0   
...                                              ...        ...         ...   
write         app#corona173#9#2414020            0.0          0     8388005   
              app#corona173#9#2414745            0.0          0     2097015   
              app#corona173#9#2415364            0.0          0     8388005   
              app#corona173#9#2415823            0.0          0     8388005   
              app#corona173#9#2416516            0.0          0     2096960   

                                       ...  seek_time  stat_time  close_count  \
func_id       proc_name                ...                                      
<module>.iter app#corona171#0#4101311  ...        0.0        0.0            0   
              app#corona171#1#4101314  ...        0.0        0.0            0   
              app#corona171#2#4101317  ...        0.0        0.0            0   
              app#corona171#3#4101319  ...        0.0        0.0            0   
              app#corona171#4#4101314  ...        0.0        0.0            0   
...                                    ...        ...        ...          ...   
write         app#corona173#9#2414020  ...        0.0        0.0            0   
              app#corona173#9#2414745  ...        0.0        0.0            0   
              app#corona173#9#2415364  ...        0.0        0.0            0   
              app#corona173#9#2415823  ...        0.0        0.0            0   
              app#corona173#9#2416516  ...        0.0 

time: 68.9 ms (started: 2024-01-25 16:46:38 -08:00)


In [12]:
result.evaluated_views['time'][('func_id',)].evaluated_groups.compute()

,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,time_sum,time_per,time_per_rev_cs,time_per_rev_cs_diff,time_slope,time_norm,time_pero,time_perr,time_score,time_threshold
func_id,,,,,,,,,,,,,,,,,,,,,
DLIOBenchmark.run,2411.926066,16,0,0,0,0.0,0.0,0.0,0,0,...,9129.133385,0.735799,0.264201,0.000000,0.000000,0.264201,0.264201,0.272088,medium,0.250
DLIOBenchmark._train,2165.608251,80,0,0,0,0.0,0.0,0.0,0,0,...,9129.133385,0.762781,0.501420,0.237219,0.051655,0.237219,0.237219,0.244301,low,0.100
<module>.iter,1903.097546,80,0,0,0,0.0,0.0,0.0,0,0,...,9129.133385,0.791536,0.709885,0.208464,0.058780,0.208464,0.208464,0.214687,low,0.100
DYADTorchDataset.__getitem__,278.068725,2558,2096704,2096704,5363368832,0.0,0.0,0.0,0,0,...,9129.133385,0.969541,0.740344,0.030459,12.653359,0.030459,0.030459,0.031369,very low,0.010
TorchFramework.init_loader,231.874346,80,0,0,0,0.0,0.0,0.0,0,0,...,9129.133385,0.974601,0.765744,0.025399,0.482421,0.025399,0.025399,0.026158,very low,0.010
TorchFramework.compute,217.873424,160,0,0,0,0.0,0.0,0.0,0,0,...,9129.133385,0.976134,0.789609,0.023866,1.026759,0.023866,0.023866,0.024578,very low,0.010
Dyad.init,206.836093,240,0,0,0,0.0,0.0,0.0,0,0,...,9129.133385,0.977343,0.812266,0.022657,1.622065,0.022657,0.022657,0.023333,very low,0.010
dyad_init,206.255181,240,0,0,0,0.0,0.0,0.0,0,0,...,9129.133385,0.977407,0.834859,0.022593,1.626631,0.022593,0.022593,0.023268,very low,0.010
dyad_dtl_init,206.198837,242,0,0,0,0.0,0.0,0.0,0,0,...,9129.133385,0.977413,0.857446,0.022587,1.640627,0.022587,0.022587,0.023261,very low,0.010


time: 142 ms (started: 2024-01-25 16:27:58 -08:00)


In [9]:
result.output.console()

KeyError: 'func_id'

time: 32.4 s (started: 2024-01-25 16:26:31 -08:00)


In [7]:
result.output.console()

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│  Job Time             1706065067063043.00 seconds                                                               │
│  I/O Time             1706065062326307.00 seconds                                                               │
│                       ├── Read - 3680889.00 seconds (0.00%)                                                     │
│                       ├── Write - 194985.00 seconds (0.00%)                                                     │
│                       └── Metadata - 616583.00 seconds (0.00%)                                                  │
│  I/O Ops              374,312 ops                                                                               │
│                       ├── Read - 67,754 ops (18.10%)                                                            │
│                       ├── Write - 1,086 ops (0.29%)                                                             │
│                       └── Metadata - 221,506 ops (59.18%)                                                       │
│  I/O Size             38.96 GiB                                                                                 │
│                       ├── Read - 37.73 GiB (96.84%)                                                             │
│                       └── Write - 1.23 GiB (3.16%)                                                              │
│  Read Xfer            4 kiB-4 MiB - 19,204 ops                                                                  │
│                       ├── <4 kiB - 2,838 ops (14.78%)                                                           │
│                       ├── 4-16 kiB - 30 ops (0.16%)                                                             │
│                       ├── 64-256 kiB - 7,682 ops (40.00%)                                                       │
│                       ├── 256 kiB-1 MiB - 30 ops (0.16%)                                                        │
│                       └── 1-4 MiB - 8,624 ops (44.91%)                                                          │
│  Write Xfer           4 kiB->64 MiB - 625 ops                                                                   │
│                       ├── <4 kiB - 242 ops (38.72%)                                                             │
│                       ├── 1-4 MiB - 381 ops (60.96%)                                                            │
│                       ├── 16-64 MiB - 1 ops (0.16%)                                                             │
│                       └── >64 MiB - 1 ops (0.16%)                                                               │
│  Node(s)              2 node(s)                                                                                 │
│                       ├── corona173 - 1706065062326307.00 s (100.00%) - 19.69 GiB/375.97 MiB R/W (50.54/0.94%)  │
│                       │   - 190,733 ops (50.96%)                                                                │
│                       └── corona171 - 1706065061711003.00 s (100.00%) - 18.04 GiB/885.61 MiB R/W (46.30/2.22%)  │
│                           - 183,579 ops (49.04%)                                                                │
│  App(s)               1 app(s)                                                                                  │
│                       └── app - 1706065062326307.00 s (100.00%) - 37.73 GiB/1.23 GiB R/W (96.84/3.16%) -        │
│                           374,312 ops (100.00%)                                                                 │
│  Process(es)/Rank(s)  498 process(es)/rank(s)                                                                   │
│  Files                344 files                                                                                 │
│                       ├── Shared: 342 files (99.42%)  

time: 32 s (started: 2024-01-25 15:31:59 -08:00)


In [7]:
traces, main_view = result

time: 892 µs (started: 2024-01-25 15:29:16 -08:00)


In [8]:
traces.dtypes

func_id         string[pyarrow]
cat             string[pyarrow]
pid                       int64
tid                       int64
ts                        int64
te                        int64
time                      int64
time_range              float64
host_name       string[pyarrow]
compute_time              int64
io_time                   int64
app_io_time               int64
total_time                int64
file_name       string[pyarrow]
phase                     int16
size                      int64
proc_name       string[pyarrow]
acc_pat                   int64
count                     int64
io_cat                    int64
dtype: object

time: 5.5 ms (started: 2024-01-25 15:29:17 -08:00)


In [10]:
main_view.dtypes

time                  int64
count                 int64
size_min              int64
size_max              int64
size                  int64
read_time           float64
write_time          float64
metadata_time       float64
read_size             int64
write_size            int64
metadata_size         int64
read_count            int64
write_count           int64
metadata_count        int64
read_min              int64
read_max              int64
write_min             int64
write_max             int64
metadata_min          int64
metadata_max          int64
data_count            int64
data_size             int64
data_time           float64
sequential_time     float64
random_time         float64
sequential_size       int64
random_size           int64
sequential_count      int64
random_count          int64
close_time          float64
open_time           float64
seek_time           float64
stat_time           float64
close_count           int64
open_count            int64
seek_count          

time: 5.62 ms (started: 2024-01-25 15:29:39 -08:00)


In [9]:
t = traces.compute()
t

,func_id,cat,pid,tid,ts,te,time,time_range,host_name,compute_time,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat
0,dyad_ucx_ep_cache_init,C_APP,727134,1454268,1706065017993766,1706065017993771,5,1.706065e+12,corona173,0,0,0,0,,0,0,app#corona173#727134#1454268,0,1,0
1,ucx_allocate_buffer,C_APP,727134,1454268,1706065017993820,1706065018782713,788893,1.706065e+12,corona173,0,0,0,0,,0,0,app#corona173#727134#1454268,0,1,0
2,dyad_dtl_ucx_get_buffer,C_APP,727134,1454268,1706065018782778,1706065018782781,3,1.706065e+12,corona173,0,0,0,0,,0,0,app#corona173#727134#1454268,0,1,0
3,ucx_connect,C_APP,727134,1454268,1706065018782794,1706065018819568,36774,1.706065e+12,corona173,0,0,0,0,,0,0,app#corona173#727134#1454268,0,1,0
4,dyad_ucx_request_init,C_APP,727134,1454268,1706065018819620,1706065018819622,2,1.706065e+12,corona173,0,0,0,0,,0,0,app#corona173#727134#1454268,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,TorchFramework.checkpoint,ai_framework,4,3552674,1706065063121917,1706065063121960,43,1.706065e+12,corona171,0,0,0,0,,0,0,app#corona171#4#3552674,0,1,0
351,DLIOBenchmark._train,dlio_benchmark,4,3552674,1706065037517196,1706065063123608,25606412,1.706065e+12,corona171,0,0,0,0,,0,0,app#corona171#4#3552674,0,1,0
352,TorchFramework.get_loader,ai_framework,4,3552674,1706065063123768,1706065063123782,14,1.706065e+12,corona171,0,0,0,0,,0,0,app#corona171#4#3552674,0,1,0
353,DyadTorchDataLoader.finalize,data_loader,4,3552674,1706065063123831,1706065063123838,7,1.706065e+12,corona171,0,0,0,0,,0,0,app#corona171#4#3552674,0,1,0


time: 9.25 s (started: 2024-01-25 15:29:24 -08:00)


In [13]:
t[t['pid'].isna()]

,func_id,cat,pid,tid,ts,te,time,tinterval,time_range,host_name,...,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat


time: 45.7 ms (started: 2024-01-25 15:14:44 -08:00)


In [10]:
t['size'].describe()

count    3.743120e+05
mean     4.694983e+05
std      7.672694e+06
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      4.991532e+08
Name: size, dtype: float64

time: 19 ms (started: 2024-01-25 15:12:36 -08:00)


In [11]:
t[t['size'].isna()]

,func_id,cat,pid,tid,ts,te,time,tinterval,time_range,host_name,...,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat


time: 94.6 ms (started: 2024-01-25 15:12:42 -08:00)


In [34]:
mv = main_view.compute()
mv

time  \
file_name                         proc_name               time_range                
/l/ssd/haridev/dyad               app#corona171#0#3552649 1706064871320.0     246   
                                  app#corona171#0#3553042 1706064925269.0      10   
                                                          1706064940515.0       4   
                                                          1706064941514.0       5   
                                                          1706064941531.0       4   
...                                                                           ...   
checkpoints/unet3d/model-5-2-0.pt app#corona171#0#3552666 1706065063122.0    2026   
                                                          1706065063124.0      11   
                                                          1706065063284.0      57   
                                                          1706065063285.0  185492   
                                                          1706065063474.0  541815   

                                                                           count  \
file_name                         proc_name               time_range               
/l/ssd/haridev/dyad               app#corona171#0#3552649 1706064871320.0      1   
                                  app#corona171#0#3553042 1706064925269.0      2   
                                                          1706064940515.0      1   
                                                          1706064941514.0      1   
                                                          1706064941531.0      1   
...                                                                          ...   
checkpoints/unet3d/model-5-2-0.pt app#corona171#0#3552666 1706065063122.0      1   
                                                          1706065063124.0      2   
                                                          1706065063284.0      1   
                                                          1706065063285.0      1   
                                                          1706065063474.0      2   

                                                                            size_min  \
file_name                         proc_name               time_range                   
/l/ssd/haridev/dyad               app#corona171#0#3552649 1706064871320.0          0   
                                  app#corona171#0#3553042 1706064925269.0          0   
                                                          1706064940515.0          0   
                                                          1706064941514.0          0   
                                                          1706064941531.0          0   
...                                                                              ...   
checkpoints/unet3d/model-5-2-0.pt app#corona171#0#3552666 1706065063122.0          0   
                                                          1706065063124.0          0   
                                                          1706065063284.0        320   
                                                          1706065063285.0  499153191   
                                                          1706065063474.0        388   

                                                                            size_max  \
file_name                         proc_name               time_range                   
/l/ssd/haridev/dyad               app#corona171#0#3552649 1706064871320.0          0   
                                  app#corona171#0#3553042 1706064925269.0          0   
                                                          1706064940515.0          0   
                                                          1706064941514.0          0   
                                                          1706064941531.0          0   
...                                                                              ...   
checkpoints/unet3d/model-5-2-

time: 266 ms (started: 2024-01-25 15:04:28 -08:00)


In [10]:
main_view, view_results, evaluated_views = result

time: 1.42 ms (started: 2024-01-25 14:53:34 -08:00)


In [21]:
mv = main_view.compute()
mv

time  \
file_name                         proc_name               time_range                
/l/ssd/haridev/dyad               app#corona171#0#3552649 1706064871320.0     246   
                                  app#corona171#0#3553042 1706064925269.0      10   
                                                          1706064940515.0       4   
                                                          1706064941514.0       5   
                                                          1706064941531.0       4   
...                                                                           ...   
checkpoints/unet3d/model-5-2-0.pt app#corona171#0#3552666 1706065063122.0    2026   
                                                          1706065063124.0      11   
                                                          1706065063284.0      57   
                                                          1706065063285.0  185492   
                                                          1706065063474.0  541815   

                                                                           count  \
file_name                         proc_name               time_range               
/l/ssd/haridev/dyad               app#corona171#0#3552649 1706064871320.0      1   
                                  app#corona171#0#3553042 1706064925269.0      2   
                                                          1706064940515.0      1   
                                                          1706064941514.0      1   
                                                          1706064941531.0      1   
...                                                                          ...   
checkpoints/unet3d/model-5-2-0.pt app#corona171#0#3552666 1706065063122.0      1   
                                                          1706065063124.0      2   
                                                          1706065063284.0      1   
                                                          1706065063285.0      1   
                                                          1706065063474.0      2   

                                                                            size_min  \
file_name                         proc_name               time_range                   
/l/ssd/haridev/dyad               app#corona171#0#3552649 1706064871320.0          0   
                                  app#corona171#0#3553042 1706064925269.0          0   
                                                          1706064940515.0          0   
                                                          1706064941514.0          0   
                                                          1706064941531.0          0   
...                                                                              ...   
checkpoints/unet3d/model-5-2-0.pt app#corona171#0#3552666 1706065063122.0          0   
                                                          1706065063124.0          0   
                                                          1706065063284.0        320   
                                                          1706065063285.0  499153191   
                                                          1706065063474.0        388   

                                                                            size_max  \
file_name                         proc_name               time_range                   
/l/ssd/haridev/dyad               app#corona171#0#3552649 1706064871320.0          0   
                                  app#corona171#0#3553042 1706064925269.0          0   
                                                          1706064940515.0          0   
                                                          1706064941514.0          0   
                                                          1706064941531.0          0   
...                                                                              ...   
checkpoints/unet3d/model-5-2-

time: 300 ms (started: 2024-01-25 14:58:18 -08:00)


In [30]:
mv.dtypes

time                  Int64
count                 int64
size_min              Int64
size_max              Int64
size                  Int64
read_time            object
write_time           object
metadata_time        object
read_size            object
write_size           object
metadata_size        object
read_count            int64
write_count           int64
metadata_count        int64
read_min              Int64
read_max              Int64
write_min             Int64
write_max             Int64
metadata_min          Int64
metadata_max          Int64
data_count            int64
data_size            object
data_time            object
sequential_time       int64
random_time         float64
sequential_size       int64
random_size           int64
sequential_count      int64
random_count          int64
close_time           object
open_time            object
seek_time            object
stat_time            object
close_count           int64
open_count            int64
seek_count          

time: 5.69 ms (started: 2024-01-25 15:01:02 -08:00)


In [29]:
mv['read_size'].dtype

dtype('O')

time: 3.21 ms (started: 2024-01-25 15:00:14 -08:00)


In [18]:
tppb = evaluated_views['time'][('proc_name',)].potential_bottlenecks.compute()
tppb

,time,count,size_min,size_max,size,read_time,write_time,metadata_time,read_size,write_size,...,time_sum,time_per,time_per_rev_cs,time_per_rev_cs_diff,time_slope,time_norm,time_pero,time_perr,time_score,time_threshold
proc_name,,,,,,,,,,,,,,,,,,,,,
app#corona173#14#721811,10113542,5011,0,4194304,1154104599,3558160.0,5701.0,168193.0,1114265046,14679105,...,452334850,0.977641,0.022359,0.0,0.0,1.0,1.0,0.028986,critical,0.90
app#corona173#9#721814,9909621,4806,0,4194304,1099662570,3607503.0,26591.0,92249.0,1061920032,12582090,...,452334850,0.978092,0.044266,0.021908,35.794342,0.979837,0.979837,0.028401,critical,0.90
app#corona171#5#3553034,9668985,4555,0,4194304,1038047683,3518102.0,7241.0,163799.0,994014265,18872970,...,452334850,0.978624,0.065642,0.021376,35.008245,0.956043,0.956043,0.027711,critical,0.90
app#corona173#10#721864,9647595,4846,0,4194304,1120593304,3680889.0,41445.0,81726.0,1076559831,18873025,...,452334850,0.978672,0.08697,0.021328,36.753156,0.953928,0.953928,0.02765,critical,0.90
app#corona173#8#721811,9632354,5158,0,4194304,1197802768,3462664.0,19776.0,140317.0,1153769295,18873025,...,452334850,0.978705,0.108265,0.021295,38.525455,0.952421,0.952421,0.027606,critical,0.90
app#corona171#0#3553108,9535960,4983,0,6291008,1153778630,3520289.0,56872.0,147008.0,1111842117,16776065,...,452334850,0.978918,0.129347,0.021082,37.844506,0.94289,0.94289,0.02733,critical,0.90
app#corona171#0#3553042,9118455,4045,0,4194304,906860735,3540516.0,34664.0,81597.0,867021292,14678995,...,452334850,0.979841,0.149505,0.020159,33.406959,0.901608,0.901608,0.026134,critical,0.90
app#corona171#3#3553048,9003238,4605,0,4194304,1053095627,3362815.0,35166.0,84644.0,1011159169,16776010,...,452334850,0.980096,0.169409,0.019904,37.251971,0.890216,0.890216,0.025803,very high,0.75
app#corona173#11#721820,9000439,4919,0,4194304,1132476883,3184892.0,21303.0,118076.0,1090540370,16776065,...,452334850,0.980102,0.189307,0.019898,39.096575,0.889939,0.889939,0.025795,very high,0.75


time: 63 ms (started: 2024-01-25 14:56:19 -08:00)


In [20]:
tppb[['count', 'read_size', 'write_size']]

,count,read_size,write_size
proc_name,,,
app#corona173#14#721811,5011,1114265046,14679105
app#corona173#9#721814,4806,1061920032,12582090
app#corona171#5#3553034,4555,994014265,18872970
app#corona173#10#721864,4846,1076559831,18873025
app#corona173#8#721811,5158,1153769295,18873025
app#corona171#0#3553108,4983,1111842117,16776065
app#corona171#0#3553042,4045,867021292,14678995
app#corona171#3#3553048,4605,1011159169,16776010
app#corona173#11#721820,4919,1090540370,16776065


time: 12.8 ms (started: 2024-01-25 14:56:56 -08:00)


In [60]:
df = result.compute()
df

,func_id,cat,pid,tid,ts,te,time,tinterval,time_range,host_name,...,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat
0,dyad_ucx_ep_cache_init,C_APP,727134,1454268,1706065017993766,1706065017993771,5,<NA>,1.706065e+12,corona173,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona173#727134#1454268,0,1,0
1,ucx_allocate_buffer,C_APP,727134,1454268,1706065017993820,1706065018782713,788893,<NA>,1.706065e+12,corona173,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona173#727134#1454268,0,1,0
2,dyad_dtl_ucx_get_buffer,C_APP,727134,1454268,1706065018782778,1706065018782781,3,<NA>,1.706065e+12,corona173,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona173#727134#1454268,0,1,0
3,ucx_connect,C_APP,727134,1454268,1706065018782794,1706065018819568,36774,<NA>,1.706065e+12,corona173,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona173#727134#1454268,0,1,0
4,dyad_ucx_request_init,C_APP,727134,1454268,1706065018819620,1706065018819622,2,<NA>,1.706065e+12,corona173,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona173#727134#1454268,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,TorchFramework.checkpoint,ai_framework,4,3552674,1706065063121917,1706065063121960,43,<NA>,1.706065e+12,corona171,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona171#4#3552674,0,1,0
351,DLIOBenchmark._train,dlio_benchmark,4,3552674,1706065037517196,1706065063123608,25606412,<NA>,1.706065e+12,corona171,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona171#4#3552674,0,1,0
352,TorchFramework.get_loader,ai_framework,4,3552674,1706065063123768,1706065063123782,14,<NA>,1.706065e+12,corona171,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona171#4#3552674,0,1,0
353,DyadTorchDataLoader.finalize,data_loader,4,3552674,1706065063123831,1706065063123838,7,<NA>,1.706065e+12,corona171,...,<NA>,<NA>,0,<NA>,0,<NA>,app#corona171#4#3552674,0,1,0


time: 7.17 s (started: 2024-01-24 21:51:19 -08:00)


In [61]:
df[~df['io_time'].isna()]

,func_id,cat,pid,tid,ts,te,time,tinterval,time_range,host_name,...,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat
2,readlink,POSIX,8,725367,1706064990072902,1706064990072908,6,<NA>,1.706065e+12,corona173,...,6,<NA>,6,/l/ssd/haridev/dyad,2,<NA>,app#corona173#8#725367,0,1,0
3,readlink,POSIX,8,725367,1706064990073044,1706064990073047,3,<NA>,1.706065e+12,corona173,...,3,<NA>,3,/l/ssd/haridev/dyad,2,<NA>,app#corona173#8#725367,0,1,0
8,readlink,POSIX,8,725367,1706065002488397,1706065002488402,5,<NA>,1.706065e+12,corona173,...,5,<NA>,5,/l/ssd/haridev/dyad,2,<NA>,app#corona173#8#725367,0,1,0
9,readlink,POSIX,8,725367,1706065002488450,1706065002488454,4,<NA>,1.706065e+12,corona173,...,4,<NA>,4,/l/ssd/haridev/dyad/img_050_of_168.npz,2,<NA>,app#corona173#8#725367,0,1,0
10,open,POSIX,8,725367,1706065002488578,1706065002488590,12,<NA>,1.706065e+12,corona173,...,12,<NA>,12,/l/ssd/haridev/dyad/img_050_of_168.npz,2,<NA>,app#corona173#8#725367,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,__xstat64,POSIX,4,3552674,1706064980642035,1706064980642039,4,<NA>,1.706065e+12,corona171,...,4,<NA>,4,checkpoints/unet3d,2,<NA>,app#corona171#4#3552674,0,1,3
299,mkdir,POSIX,4,3552674,1706065009223221,1706065009223745,524,<NA>,1.706065e+12,corona171,...,524,<NA>,524,checkpoints/unet3d,2,<NA>,app#corona171#4#3552674,0,1,3
300,__xstat64,POSIX,4,3552674,1706065009223810,1706065009223814,4,<NA>,1.706065e+12,corona171,...,4,<NA>,4,checkpoints/unet3d,2,<NA>,app#corona171#4#3552674,0,1,3
329,mkdir,POSIX,4,3552674,1706065037516410,1706065037517018,608,<NA>,1.706065e+12,corona171,...,608,<NA>,608,checkpoints/unet3d,2,<NA>,app#corona171#4#3552674,0,1,3


time: 84.7 ms (started: 2024-01-24 21:51:38 -08:00)


In [62]:
df[~df['size'].isna()]

,func_id,cat,pid,tid,ts,te,time,tinterval,time_range,host_name,...,io_time,app_io_time,total_time,file_name,phase,size,proc_name,acc_pat,count,io_cat
20,read,POSIX,8,725367,1706065002488956,1706065002488963,7,<NA>,1.706065e+12,corona173,...,7,<NA>,7,/l/ssd/haridev/dyad/img_050_of_168.npz,2,4096,app#corona173#8#725367,0,1,1
26,read,POSIX,8,725367,1706065002489175,1706065002489181,6,<NA>,1.706065e+12,corona173,...,6,<NA>,6,/l/ssd/haridev/dyad/img_050_of_168.npz,2,22,app#corona173#8#725367,0,1,1
27,read,POSIX,8,725367,1706065002489195,1706065002489199,4,<NA>,1.706065e+12,corona173,...,4,<NA>,4,/l/ssd/haridev/dyad/img_050_of_168.npz,2,0,app#corona173#8#725367,0,1,1
29,read,POSIX,8,725367,1706065002489246,1706065002489251,5,<NA>,1.706065e+12,corona173,...,5,<NA>,5,/l/ssd/haridev/dyad/img_050_of_168.npz,2,42,app#corona173#8#725367,0,1,1
31,read,POSIX,8,725367,1706065002489294,1706065002489298,4,<NA>,1.706065e+12,corona173,...,4,<NA>,4,/l/ssd/haridev/dyad/img_050_of_168.npz,2,73,app#corona173#8#725367,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,read,POSIX,4,3560981,1706065060849022,1706065060849074,52,<NA>,1.706065e+12,corona171,...,52,<NA>,52,/l/ssd/haridev/dyad/img_021_of_168.npz,2,258048,app#corona171#4#3560981,0,1,1
534,read,POSIX,4,3560981,1706065060849093,1706065060849097,4,<NA>,1.706065e+12,corona171,...,4,<NA>,4,/l/ssd/haridev/dyad/img_021_of_168.npz,2,4096,app#corona171#4#3560981,0,1,1
536,read,POSIX,4,3560981,1706065060849367,1706065060849424,57,<NA>,1.706065e+12,corona171,...,57,<NA>,57,/l/ssd/haridev/dyad/img_021_of_168.npz,2,253952,app#corona171#4#3560981,0,1,1
537,read,POSIX,4,3560981,1706065060849442,1706065060849447,5,<NA>,1.706065e+12,corona171,...,5,<NA>,5,/l/ssd/haridev/dyad/img_021_of_168.npz,2,3904,app#corona171#4#3560981,0,1,1


time: 59.2 ms (started: 2024-01-24 21:51:58 -08:00)


In [35]:
df['time_range'].nunique()

14805

time: 4.74 ms (started: 2024-01-24 21:33:09 -08:00)


In [40]:
df[df['cat'] == 'POSIX']['func_id'].unique()

<ArrowStringArray>
[  'readlink',       'open',      'fcntl',      'lseek',      'close',
     'open64', '__fxstat64',    'lseek64',       'read',    '__xstat',
      'write',      'fsync',    'opendir',  '__xstat64',      'mkdir']
Length: 15, dtype: string

time: 56.9 ms (started: 2024-01-24 21:37:02 -08:00)


In [58]:
df.groupby(['io_cat']).agg({'func_id': 'unique'})

,func_id
io_cat,
0,"[dyad_ucx_ep_cache_init, ucx_allocate_buffer, ..."
1,[read]
2,[write]
3,"[open, fcntl, lseek, close, open64, __fxstat64..."


time: 29.6 ms (started: 2024-01-24 21:49:38 -08:00)
